calculating scores from the sampled_paramet.xlsx files output by climraster

In [60]:
import pandas as pd
import xarray as xr
import openpyxl as xl
import os

In [61]:
file = r"C:\Users\CAMG038492\OneDrive - WSP O365\Documents\Climate Data\John Cabot GIS\Output Samples\sampled_sfcWindmax.xlsx"
wb = xl.load_workbook(file)

paramet = 'sfcWindmax'

outputfolder = r"C:\Users\CAMG038492\OneDrive - WSP O365\Documents\Climate Data\John Cabot GIS\Output Samples"
outputfile = f"zscores_{paramet}.csv"
loc = os.path.join(outputfolder,outputfile)

wb.sheetnames

['1976-2005', '2006-2035', '2021-2050', '2051-2080', '2071-2100']

In [62]:
sheets = pd.read_excel(file, engine="openpyxl", sheet_name=None)

In [63]:
def reshape(ssheet):
    sections = {} # dictionary of dictionaries
    for item in ssheet.columns:
        d = {} # dictionary
        if item.startswith("sfc"):
            na,me = item.split("_ann")
            func = me.split("_")[0]
            d[func] = ssheet[item].values[0]
            try:
                sections[na] = sections[na] | d
            except KeyError:
                sections[na] = d
            if 'sfcWindmax-anwindspeed63' in na.split("_"): d['thr'] = 30.0/365
            elif 'sfcWindmax-anwindspeed90' in na.split("_"): d['thr'] = 2.0/365
            else: d['thr'] = 0
    return sections

d = {}
for years in sheets.keys():
    d = d | reshape(sheets[years])
d

{'sfcWindmax-anwindgust63_CanRCM4_historical_1976-2005': {'max': np.int64(1),
  'thr': 0,
  'mean': np.float64(0.5410330761943061),
  'sum': np.float64(197.4770728109217)},
 'sfcWindmax-anwindgust90_CanRCM4_historical_1976-2005': {'max': np.int64(1),
  'thr': 0,
  'mean': np.float64(0.1406803298515698),
  'sum': np.float64(51.34832039582295)},
 'sfcWindmax-anwindspeed63_CanRCM4_historical_1976-2005': {'max': np.int64(1),
  'thr': 0.0821917808219178,
  'mean': np.float64(0.1372758266771131),
  'std': np.float64(0.3423098118718721),
  'sum': np.float64(50.10567673714631)},
 'sfcWindmax-anwindspeed90_CanRCM4_historical_1976-2005': {'max': np.float64(0.7570890037040596),
  'thr': 0.005479452054794521,
  'mean': np.float64(0.005832637738859131),
  'std': np.float64(0.06357681849878408),
  'sum': np.float64(2.128912774683583)},
 'sfcWindmax_CanRCM4_historical_1976-2005': {'max': np.float64(97.50906372070312),
  'thr': 0,
  'mean': np.float64(46.28534317016602),
  'std': np.float64(15.2775688

In [ ]:
import scipy.stats
def z2p(z):
    p_value = scipy.stats.norm.sf(abs(z))
    return p_value

def p2l(p):
    if p < .2 and p > 0: l = 1
    elif p < .4: l = 2
    elif p < .6: l = 3
    elif p < .8: l = 4
    elif p < 1: l = 5
    else: raise ValueError(f"invalid p value {p}")
    return l

In [ ]:
#thr = 30.0
rowdict = {}
for item in d.keys():
    try:
        mean = d[item]['mean']
    except KeyError:
        #raise ValueError("missing data")
        pass
    try:
        std = d[item]['std']
    except KeyError:
        #raise ValueError("missing data")
        pass
    try:
        thr = d[item]['thr']
    except KeyError:
        #raise ValueError("missing data")
        pass
    try:
        zscore = (thr - mean) / std
        pvalue = z2p(zscore)
        invpvalue = z2p(1-zscore)
        score = p2l(pvalue)
        invscore = p2l(invpvalue)
        rowdict[item] = {'mean' : mean, 'std' : std, 'thr' : thr, 'zscore' : zscore,
                         'mean days' : mean*365, 'std days' : std*365,
                         'p value' : pvalue, 'inverted p value' : invpvalue,
                         'climate score' : score, 'inv climate score' : invscore}
    except:
        pass

In [66]:
sdf = pd.DataFrame(rowdict).transpose()
sdf

,mean,std,thr,zscore,mean days,std days,p value,inverted p value,climate score,inv climate score
sfcWindmax-anwindgust63_CanRCM4_historical_1976-2005,0.541033,15.255819,0.000000,-0.035464,197.477073,5568.374052,0.485855,0.150226,3.0,1.0
sfcWindmax-anwindgust90_CanRCM4_historical_1976-2005,0.140680,15.255819,0.000000,-0.009221,51.348320,5568.374052,0.496321,0.156434,3.0,1.0
sfcWindmax-anwindspeed63_CanRCM4_historical_1976-2005,0.137276,0.342310,0.082192,-0.160919,50.105677,124.943081,0.436079,0.122837,3.0,1.0
sfcWindmax-anwindspeed90_CanRCM4_historical_1976-2005,0.005833,0.063577,0.005479,-0.005555,2.128913,23.205539,0.497784,0.157315,3.0,1.0
sfcWindmax_CanRCM4_historical_1976-2005,46.285343,15.277569,0.000000,-3.029628,16894.150257,5576.312618,0.001224,0.000028,1.0,1.0
sfcWindmax-anwindgust63_CanRCM4_rcp45_2006-2035,0.545990,15.277569,0.000000,-0.035738,199.286317,5576.312618,0.485746,0.150162,3.0,1.0
sfcWindmax-anwindgust90_CanRCM4_rcp45_2006-2035,0.142515,15.277569,0.000000,-0.009328,52.017870,5576.312618,0.496279,0.156409,3.0,1.0
sfcWindmax-anwindspeed63_CanRCM4_rcp45_2006-2035,0.137860,0.342819,0.082192,-0.162383,50.318749,125.128762,0.435502,0.122540,3.0,1.0
sfcWindmax-anwindspeed90_CanRCM4_rcp45_2006-2035,0.006818,0.072870,0.005479,-0.018372,2.488642,26.597697,0.492671,0.154251,3.0,1.0
sfcWindmax_CanRCM4_rcp45_2006-2035,46.408218,15.282841,0.000000,-3.036623,16938.999710,5578.236866,0.001196,0.000027,1.0,1.0


In [67]:
sdf.to_csv(loc)